In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from PIL import Image
from tensorflow.keras.utils import to_categorical

In [ ]:
data_dir = "/kaggle/input/Kannada-MNIST/"
model_dir = '/kaggle/input/mobilenet-v2-tf-kernels-96-no-top/mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.0_96_no_top.h5'

In [ ]:
# data_dir = "/kaggle/input/Kannada-MNIST/"
train = pd.read_csv(data_dir + "train.csv")
train.shape

In [ ]:
test = pd.read_csv(data_dir + "test.csv")
test.shape

In [ ]:
X_train_ = train.iloc[:,1:]
y_train_ = train.iloc[:,0]
X_test_ = test.iloc[:,1:]

In [ ]:
def oneD_to_threeD(images, labels, row_number, shape=28):
    images = images.iloc[0:row_number,:].values #取出部分数据（此处500）并转为uint8格式方便CV处理
    labels = labels.iloc[0:row_number]
    train_data = []
    for i in range(row_number):
        img = images[i,:]
        img = img.reshape(28, 28) #reshape为2维矩阵
        img = Image.fromarray(img.astype('uint8')).convert('RGB') #转化为image方便resize并转化为三通道图像
        resized_img = img.resize((shape,shape), Image.ANTIALIAS) #resize到需要的尺寸
        resized_img = np.array(resized_img).astype('float32') #重新转化为array以输入神经网络
        train_data.append(resized_img)
    train_data=np.array(train_data).astype('float32') #转为float32格式方便神经网络识别
    labels = to_categorical(labels) #标签化label
    return train_data, labels

In [ ]:
X_train, y_train= oneD_to_threeD(X_train_, y_train_, 20000, 96)

In [ ]:
X_test, _ = oneD_to_threeD(X_test_, pd.DataFrame(range(len(X_test_))), len(X_test_), 96)

In [ ]:
X_train = X_train/255 
X_test = X_test/255

In [ ]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Dropout, MaxPooling2D, GlobalAveragePooling2D

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import cv2
from tensorflow import keras

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_vali, y_train, y_vali = train_test_split(X_train,
                                                   y_train,
                                                   test_size = 0.2)

print(X_train.shape, X_vali.shape, y_train.shape, y_vali.shape)

In [ ]:
# reconstructed_model = keras.models.load_model(model_dir)

In [ ]:
base_model = keras.applications.MobileNetV2(weights = model_dir, input_shape = (96, 96, 3), include_top = False)

In [ ]:
base_model.trainable = False

In [ ]:
model = Sequential([
    base_model,
    GlobalAveragePooling2D(), 
    Dropout(0.4), 
    Dense(10, activation = 'softmax')])

model.summary()

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate = 0.001, beta_1 = 0.9, beta_2 = 0.999)
model.compile(optimizer=optimizer,
              loss=['categorical_crossentropy'],
              metrics=['accuracy'])

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1)

In [ ]:
datagen.fit(X_train)

In [ ]:
epochs = 30
batch_size = 64
history = model.fit(datagen.flow(X_train, y_train, batch_size=batch_size), epochs=epochs, 
                    validation_data=(X_vali, y_vali))

In [ ]:
base_model.trainable = True
model.compile(
    optimizer = keras.optimizers.Adam(1e-5),
    loss = ['categorical_crossentropy'],
    metrics = ['accuracy'],
    )
model.summary()

epochs_fine = 30
model.fit(datagen.flow(X_train, y_train, batch_size=batch_size), epochs=epochs_fine, 
                    validation_data=(X_vali, y_vali))

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
# plt.ylim([0.5, 1])
plt.legend(loc='lower right')

In [ ]:
predicted = model.predict(X_test)
predicted = np.argmax(predicted, axis = 1)

index = np.array(test.index)
res = pd.DataFrame(np.concatenate([index[:,np.newaxis],predicted[:,np.newaxis]], axis = 1), columns = ['id', 'label'])

res.to_csv('submission.csv', index = False)